In [4]:
import numpy as np

Esta función utiliza el algoritmo  de Burg para determinar los coeficientes de un modelo AR (autoregresivo) de orden p de la Aproximación de Máxima Entropía para la señal "data", una estimación de la señal dada realizando predicciones hacia adelante y atrás de orden p y los residuos de la predicción. Si se define B = burg(data, p) entonces B[0] serán los coeficientes del modelo, B[1] será la predicción lineal de la señal y B[2] serán los residuos

In [5]:
def burg(data,p):
    N = len(data)
    r = np.correlate(data-np.mean(data),data-np.mean(data),'full')/(N*np.std(data)**2)
    R = np.flip(r[0:N])
    E = np.zeros(p+1); k = np.zeros(p+1); Coef = np.zeros((p+1,p+1)); b = np.zeros((p+1,N+1))
    e = np.zeros((p+1,N+1)); E[0] = np.sum(data**2)/N
    for q in np.arange(1,N):
        b[0,q] = data[q]
        e[0,q] = data[q]
    for m in np.arange(1,p+1):
        be = 0
        be2 = 0
        for j in np.arange(m+1,N+1):
            be = be + b[m-1,j-1] * e[m-1,j]
            be2 = be2 + (b[m-1,j-1]**2 + e[m-1,j]**2)
        k[m] = -2 * be/be2
        E[m] = (1-k[m]**2)*E[m-1]
        Coef[m,m] = k[m]
        for i in np.arange(1,m):
            Coef[m,i] = Coef[m-1,i] + Coef[m,m] * Coef[m-1,m-i]
        for n in np.arange(1,N-m+1):
            e[m,n] = e[m-1,n] + Coef[m,m] * b[m-1,n-1]
            b[m,n] = b[m-1,n-1] + Coef[m,m] * e[m-1,n]
    coef = Coef[p,:]
    dataEstim = np.zeros(N); dataEstim[0] = 0
    for i in np.arange(1,N):
        s = 0
        for j in np.arange(1,p):
            if i - j >= 0:
                s = s - coef[j] * data[i-j]
        dataEstim[i] = s
    Residuos = data - dataEstim
    return coef[1:p+1], dataEstim, Residuos